In [150]:

!sudo apt install sqlite3
!cp $SQLITE_STORE_DIR/faiss_store.db .
!ls -lh faiss_store.db

counter = !cat counter.txt
counter = int(counter[0]) - 1
%env MEMORY_BANK_NAME=bank_{counter}


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sqlite3 is already the newest version (3.37.2-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
-rw-r--r-- 1 ubuntu ubuntu 203M Dec  8 04:21 faiss_store.db
env: MEMORY_BANK_NAME=bank_32


In [151]:
%%bash
echo "select value from kvstore where key = 'faiss_index:v1::${MEMORY_BANK_NAME}';" \
  | sqlite3 faiss_store.db \
  > faiss_index.json


In [122]:
!ls -lh faiss_index.json

-rw-rw-r-- 1 ubuntu ubuntu 14M Dec  7 23:40 faiss_index.json


In [153]:
import json
from IPython.display import display

# Load into dict
faiss_index = json.load(open('faiss_index.json'))
display(faiss_index.keys())
display(len(faiss_index['id_by_index']))
display(len(faiss_index['chunk_by_index']))
display(len(faiss_index['faiss_index']))

# Squad: 252 so mostly all contexts are a single chunk
# Quac: has 527 chunks so about 2 chunks per context
# NarrativeQA: 274 chunks so about 2.4 chunks per context

dict_keys(['id_by_index', 'chunk_by_index', 'faiss_index'])

238

238

12187100

In [154]:
import os
from llama_stack_client import LlamaStackClient

# Test run
memory_bank_name = os.getenv("MEMORY_BANK_NAME")

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

response = client.memory.query(
    bank_id=memory_bank_name,
    query=["Hello World"],
    params={"max_chunks": 10},
)

display(response)

for chunk, score in zip(response.chunks, response.scores):
    print(chunk)
    print(score)


QueryDocumentsResponse(chunks=[Chunk(content='He counsels them: when Hi and Ed say that they are splitting up, he advises them to sleep on it. Hi and Ed go to sleep in the same bed, and Hi has a dream about Gale and Evelle reforming after returning to prison; Glen gets his due from a Polish-American police officer after "telling one Polack joke too many"; and Nathan Jr. gets a football for Christmas from "a kindly couple who wish to remain unknown", later becoming a football star. The dream ends with an elderly couple (implied to be Hi and Ed) together enjoying a holiday visit from a large family of children and grandchildren.', document_id='20', token_count=130), Chunk(content="Hiccup confronts Drago and a brainwashed Toothless while the other riders work to distract the Bewilderbeast. Hiccup succeeds in freeing Toothless from the Bewilderbeast's control, much to Drago's surprise. Hiccup and Toothless briefly separate Drago from the Bewilderbeast and confront Drago on the ground, but 

Chunk(content='He counsels them: when Hi and Ed say that they are splitting up, he advises them to sleep on it. Hi and Ed go to sleep in the same bed, and Hi has a dream about Gale and Evelle reforming after returning to prison; Glen gets his due from a Polish-American police officer after "telling one Polack joke too many"; and Nathan Jr. gets a football for Christmas from "a kindly couple who wish to remain unknown", later becoming a football star. The dream ends with an elderly couple (implied to be Hi and Ed) together enjoying a holiday visit from a large family of children and grandchildren.', document_id='20', token_count=130)
0.671930649735797
Chunk(content="Hiccup confronts Drago and a brainwashed Toothless while the other riders work to distract the Bewilderbeast. Hiccup succeeds in freeing Toothless from the Bewilderbeast's control, much to Drago's surprise. Hiccup and Toothless briefly separate Drago from the Bewilderbeast and confront Drago on the ground, but the Bewilderbe

In [155]:
import pandas as pd

datasets = {
    "squad": "data/squad_250.parquet",
    "narrativeqa": "data/narrativeqa.parquet",
}

df = pd.read_parquet(datasets["narrativeqa"])
display(df.iloc[87]["questions"])
display(df.iloc[87]["context"])
df.head()

array(['How did Anthony (Buck Rogers) remain asleep for 492 years?',
       'What was Wilma Deering doing when she met Buck Rogers?',
       "Rogers destroyed what gang's airship?",
       'In what war did Rogers learn his battle knowledge?',
       "What did Rogers do on the day after he shot down several of the Han's airships?",
       "In which city were the records of the Han's traitorous transactions?",
       'Which gang was the traitors?',
       'Who attacked Wilma Deering and the boy when they were on air patrol?',
       'When Rogers proved himself to the Americans, what did they do?',
       'In what year did Rogers awaken from his deep slumber?',
       'What War is this story a recollection of?',
       'What company did Buck Rodgers work for in 1927?',
       'Where in Pennsylvania was Buck Rodgers investigating reports of phenomena?',
       'What was Rodgers exposed to while investigating?',
       'How many years did Rodgers remain in "sleep"?',
       'Who was the wom

' The main character and the narrator in Armageddon 2419 A.D. is Anthony Rogers, who later appears in the various comic strips, radio shows, and film serials that follow as "Buck Rogers". Rogers recounts the events of the “Second War of Independence” that precedes the first victory of Americans over Hans, in which he plays an important role.\nBorn in 1898, he was a veteran of the Great War (World War I) and was by 1927 working for the American Radioactive Gas Corporation. He was investigating reports of unusual phenomena in abandoned coal mines near Wyoming Valley in Pennsylvania. On December 15, while investigating one of the lower levels of a mine, there was a cave-in. Exposed to radioactive gas, Rogers fell into "a state of suspended animation, free from the ravages of katabolic processes, and without any apparent effect on physical or mental faculties." Rogers remained in “sleep” for 492 years.\nHe awakes in 2419 and, thinking that he has been asleep for just several hours, wanders

,context,questions
0,"""Goblin Market"" is about two close sisters, L...","[What are the sister's names?, Who lingers at ..."
1,"(From Conan The Warrior, ISBN 0-441-11465-2)\...","[How old is Conan?, Why does Conan head off to..."
2,(Note: The following synopsis was that of Emm...,"[Who is the senior clerk at James How & Sons?,..."
3,"A British woman, Elise Clifton-Ward (Angelina...","[Why is Elise being followed by the Police?, W..."
4,A group of grifters rip off their latest mark...,"[Why was one of the four grifters shot?, How d..."


In [156]:
doc_ids = []
questions = []
for _, row in df.iterrows():
    for question in row["questions"]:
        questions.append(question)
        doc_ids.append(_)

print(len(questions))
print(len(doc_ids))
display(questions[:5])
doc_ids[:40]

3461
3461


["What are the sister's names?",
 'Who lingers at the stream? ',
 'Who died at the beginning of winter? ',
 'What is strange about the grave in the story? ',
 "What did Laura save from the goblin's fruit? "]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [157]:
from tqdm import tqdm

top_k = 10

results = []

for i in tqdm(range(0, len(questions))):
    response = client.memory.query(
        bank_id=memory_bank_name,
        query=[questions[i]],
        params={"max_chunks": top_k},
    )
    res = [int(chunk.document_id) for chunk in response.chunks]
    results.append(res)

results

100%|████████████████████████████████████████████████████████████████████████████| 3461/3461 [10:47<00:00,  5.34it/s]


[[55, 91, 12, 59, 11, 80, 64, 79, 27, 70],
 [65, 21, 65, 71, 3, 79, 66, 7, 20, 100],
 [10, 16, 63, 16, 71, 49, 60, 63, 106, 63],
 [23, 16, 98, 32, 33, 10, 2, 16, 44, 63],
 [0, 0, 27, 27, 79, 43, 57, 102, 38, 91],
 [79, 0, 79, 79, 79, 0, 91, 55, 72, 91],
 [0, 0, 27, 27, 79, 91, 43, 28, 91, 45],
 [79, 34, 34, 55, 79, 103, 0, 90, 4, 89],
 [79, 0, 79, 79, 91, 13, 72, 59, 61, 58],
 [0, 0, 27, 27, 79, 100, 64, 43, 79, 91],
 [55, 12, 91, 80, 59, 13, 99, 60, 109, 34],
 [0, 71, 85, 111, 79, 86, 0, 55, 102, 60],
 [0, 0, 27, 27, 43, 79, 79, 102, 100, 45],
 [79, 0, 91, 91, 79, 0, 89, 79, 100, 98],
 [0, 79, 79, 13, 79, 91, 71, 79, 0, 28],
 [0, 79, 79, 28, 55, 91, 57, 91, 103, 85],
 [0, 0, 43, 27, 102, 27, 79, 79, 43, 71],
 [0, 0, 27, 27, 79, 102, 20, 38, 79, 43],
 [0, 0, 27, 79, 79, 27, 91, 79, 38, 28],
 [0, 0, 27, 79, 27, 43, 79, 57, 38, 91],
 [0, 27, 27, 0, 106, 30, 1, 78, 38, 30],
 [55, 12, 79, 71, 65, 91, 0, 77, 111, 68],
 [0, 0, 27, 27, 79, 79, 38, 91, 20, 28],
 [70, 35, 18, 35, 81, 92, 90, 12

In [168]:
import numpy as np

top_k = 3

res = np.array(results)
res = res[:, :top_k]
res.shape

(3461, 3)

In [169]:
targets = np.array(doc_ids)
display(targets.shape)
targets = np.expand_dims(targets, axis=1)
display(targets.shape)
targets = targets == res
display(targets.shape)
targets = targets.astype(float)
display(targets)

(3461,)

(3461, 1)

(3461, 3)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 0.]])

In [170]:
temp = np.sum(targets, axis=1)
temp = np.divide(temp, top_k)
precision = np.sum(temp) / len(temp)
precision

np.float64(0.4360011557353366)

In [171]:
temp = np.logical_or.reduce(targets, axis=1)
temp = temp.astype(float)
recall = np.sum(temp) / len(temp)
recall

np.float64(0.8038139266108061)

In [172]:
ranks = np.tile(np.arange(1, top_k + 1), (len(questions), 1)).astype(float)
temp = np.copy(targets)
for i in range(len(temp)):
    first_true = True
    for j in range(len(temp[i])):
        if temp[i,j]:
            if first_true:
                first_true = False
            else:
                temp[i,j] = False
reciprocal_ranks = temp / ranks
reciprocal_ranks = np.sum(reciprocal_ranks) / len(temp)
reciprocal_ranks = np.mean(reciprocal_ranks)
reciprocal_ranks

np.float64(0.7575845131464893)